In [25]:
import requests
import pandas as pd
headers = {
    'user-agent': 'my-app'
}


def main(url):
    with requests.Session() as req:
        req.headers.update(headers)
        allin = []
        for t in ['AMZN', 'GS']:
            params = {
                't': t
            }
            r = req.get(url, params=params)
            df = pd.read_html(r.content, attrs={'id': 'news-table'})[0]
            df[2] = t
            allin.append(df)
        df = pd.concat(allin, ignore_index=True)
        #print(df[2])
        df.to_csv('data.csv', index=False)

main('https://finviz.com/quote.ashx')

In [26]:
df = pd.read_csv('data.csv', usecols=[0,1,2], names=['date_time', 'title','ticker'])
# drop first row
data

,date_time,title,ticker
1,Jan-16-23 01:27PM,2 FAANG Stocks to Buy in 2023 and 1 to Avoid: ...,AMZN
2,12:45PM,Davos: PwC survey finds 'bleak CEO outlook' fo...,AMZN
3,11:36AM,2 Stocks to Benefit From Falling Inflation Gur...,AMZN
4,11:29AM,Big Tech Stocks Could Rebound Big Time. Heres ...,AMZN
5,10:34AM,Amazon Has a Partner for Enterprise Blockchain...,AMZN
...,...,...,...
202,10:33AM,Goldman Sachs sees less pain but also no gain ...,GS
203,09:00AM,"The Goldman Sachs Group, Inc.'s (NYSE:GS) Low ...",GS
204,07:45AM,Stack Capital Invests $8 Million USD Into Locu...,GS
205,07:40AM,Abu Dhabi's ADNOC working with Goldman Sachs o...,GS


In [18]:
data.iloc[2][0]  

'11:36AM'

In [28]:
data=data.reset_index(drop=True) 

In [29]:
data

,date_time,title,ticker
0,Jan-16-23 01:27PM,2 FAANG Stocks to Buy in 2023 and 1 to Avoid: ...,AMZN
1,12:45PM,Davos: PwC survey finds 'bleak CEO outlook' fo...,AMZN
2,11:36AM,2 Stocks to Benefit From Falling Inflation Gur...,AMZN
3,11:29AM,Big Tech Stocks Could Rebound Big Time. Heres ...,AMZN
4,10:34AM,Amazon Has a Partner for Enterprise Blockchain...,AMZN
...,...,...,...
201,10:33AM,Goldman Sachs sees less pain but also no gain ...,GS
202,09:00AM,"The Goldman Sachs Group, Inc.'s (NYSE:GS) Low ...",GS
203,07:45AM,Stack Capital Invests $8 Million USD Into Locu...,GS
204,07:40AM,Abu Dhabi's ADNOC working with Goldman Sachs o...,GS


In [32]:
parsed_data = []
for i, row in data.iterrows():
    title = data['title'].iloc[i]
    ticker = data['ticker'].iloc[i]
    date_data = data.iloc[i][0].split(' ')
    if len(date_data) == 1:
        time = date_data[0] 
    else:
        date = date_data[0]
        time = date_data[1]
        
    parsed_data.append([time, title, date, ticker])

clean_df = pd.DataFrame(parsed_data, columns=['time', 'ticker', 'date', 'title'])

In [33]:
clean_df

,time,ticker,date,title
0,01:27PM,2 FAANG Stocks to Buy in 2023 and 1 to Avoid: ...,Jan-16-23,AMZN
1,12:45PM,Davos: PwC survey finds 'bleak CEO outlook' fo...,Jan-16-23,AMZN
2,11:36AM,2 Stocks to Benefit From Falling Inflation Gur...,Jan-16-23,AMZN
3,11:29AM,Big Tech Stocks Could Rebound Big Time. Heres ...,Jan-16-23,AMZN
4,10:34AM,Amazon Has a Partner for Enterprise Blockchain...,Jan-16-23,AMZN
...,...,...,...,...
201,10:33AM,Goldman Sachs sees less pain but also no gain ...,Nov-29-22,GS
202,09:00AM,"The Goldman Sachs Group, Inc.'s (NYSE:GS) Low ...",Nov-29-22,GS
203,07:45AM,Stack Capital Invests $8 Million USD Into Locu...,Nov-29-22,GS
204,07:40AM,Abu Dhabi's ADNOC working with Goldman Sachs o...,Nov-29-22,GS


In [34]:
clean_df.to_csv('clean_data.csv', index=False)